In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
%cd /content/drive/MyDrive/cs679_project_compare_40/code/compare_graphs/numeric_compare

/content/drive/MyDrive/cs679_project_compare_40/code/compare_graphs/numeric_compare


In [7]:
!pip install librosa pesq jiwer

In [8]:
import librosa
import numpy as np

def compute_mcd(file1, file2):
    y1, sr1 = librosa.load(file1, sr=None)
    y2, sr2 = librosa.load(file2, sr=None)

    #Ensure same sample rate
    if sr1 != sr2:
        y2 = librosa.resample(y2, orig_sr=sr2, target_sr=sr1)

    mfcc1 = librosa.feature.mfcc(y=y1, sr=sr1)
    mfcc2 = librosa.feature.mfcc(y=y2, sr=sr1)
    if mfcc1.shape[1] > mfcc2.shape[1]:
        mfcc1 = mfcc1[:, :mfcc2.shape[1]]
    elif mfcc2.shape[1] > mfcc1.shape[1]:
        mfcc2 = mfcc2[:, :mfcc1.shape[1]]

    mcd_value = np.mean(np.sqrt(np.sum((mfcc1 - mfcc2) ** 2, axis=0)))
    return mcd_value


In [9]:
from pesq import pesq

def compute_pesq(reference_file, degraded_file, fs):
    """
    Compute the PESQ score.
    fs: sampling rate, use 16000 for wideband
    """
    ref, _ = librosa.load(reference_file, sr=fs)
    deg, _ = librosa.load(degraded_file, sr=fs)
    score = pesq(fs, ref, deg, 'wb' if fs == 16000 else 'nb')
    return score


In [10]:
from jiwer import wer

def compute_wer(transcription_original, transcription_transferred):
    error_rate = wer(transcription_original, transcription_transferred)
    return error_rate


In [11]:
!pip install pystoi

In [12]:
import soundfile as sf
from pystoi.stoi import stoi

def calculate_stoi(original_path, transferred_path):
    original_signal, fs_orig = sf.read(original_path)
    transferred_signal, fs_trans = sf.read(transferred_path)

    # Ensure both files have same sampling rate
    if fs_orig != fs_trans:
        transferred_signal = librosa.resample(transferred_signal, orig_sr=fs_trans, target_sr=fs_orig)
        # raise ValueError("Sampling rates do not match. STOI requires both signals to have the same sampling rate.")
    #Check if both signals have same length; pad with zeros if not
    if len(original_signal) != len(transferred_signal):
        max_len = max(len(original_signal), len(transferred_signal))
        original_signal = np.pad(original_signal, (0, max_len - len(original_signal)), 'constant')
        transferred_signal = np.pad(transferred_signal, (0, max_len - len(transferred_signal)), 'constant')

    #STOI score
    stoi_score = stoi(original_signal, transferred_signal, fs_orig, extended=False)

    return stoi_score

In [13]:
import soundfile as sf
def calculate_mse(original_path, transferred_path):
    original_signal, fs_orig = sf.read(original_path)
    transferred_signal, fs_trans = sf.read(transferred_path)

    # Ensure both files have same sampling rate
    if fs_orig != fs_trans:
        transferred_signal = librosa.resample(transferred_signal, orig_sr=fs_trans, target_sr=fs_orig)
    #Check if both signals have same length
    if len(original_signal) != len(transferred_signal):
        max_len = max(len(original_signal), len(transferred_signal))
        original_signal = np.pad(original_signal, (0, max_len - len(original_signal)), 'constant')
        transferred_signal = np.pad(transferred_signal, (0, max_len - len(transferred_signal)), 'constant')
    #Mean Squared Error
    mse = np.mean((original_signal - transferred_signal) ** 2)

    return mse

In [14]:
!pip install pyAudioAnalysis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyAudioAnalysis: filename=pyAudioAnalysis-0.3.14-py3-none-any.whl size=41264373 sha256=9cb6e15183bff7640d0c90a2c71e739d8c32544e854eec338eb0f7c6f8cff9c6
  Stored in directory: /root/.cache/pip/wheels/a7/54/73/fa830689c2440d2c81ff175c60e374930ad1607a8881e0f43f
Successfully built pyAudioAnalysis


# Methods

**MCD**: Mel-Cepstral Distortion (MCD)

**PESQ**: Perceptual Evaluation of Speech Quality (PESQ)

**WER**: Word Error Rate (WER)

**STOI**: The Short-Time Objective Intelligibility (STOI)

**MSE**: Mean Square Error

## Description

**MCD** is a measure of the spectral difference between two audio signals, typically used to assess the quality of voice conversion systems. Lower values indicate higher similarity between the audio samples.


**PESQ** scores range from -0.5 to 4.5, where higher scores indicate better speech quality. Scores above 3 are generally considered good, indicating high perceptual quality, while scores below 2 suggest poor quality, with noticeable distortions or degradation.


**WER** is a measure of the transcription error rate, calculated as the ratio of the number of errors (insertions, deletions, substitutions) to the total number of words spoken. WER values range from 0 to 1 (or 0% to 100%), where 0 indicates perfect transcription with no errors, and higher values indicate more errors.


**STOI** measures the intelligibility of speech signals and is particularly useful in evaluating the clarity of speech after style transfer. Compute the STOI score between the original and the style-transferred speech. Higher scores indicate better intelligibility.

In practical terms:

Above 0.75: Generally considered good intelligibility. Listeners are likely to understand the speech without significant effort.
0.6 to 0.75: Moderate intelligibility. While listeners can understand the speech, they might need to pay more attention, and some parts of the speech might be harder to comprehend.
Below 0.6: Poor intelligibility. Listeners may struggle to understand the speech, and there could be significant distortion or loss of information.

**MSE** quantifies the average squared difference between the values of the original and transferred signals, essentially measuring the variance between these two datasets

# Make a table

In [22]:
import pandas as pd
def main(original_file, transferred_file):
    # Compute MCD
    mcd_value = compute_mcd(original_file, transferred_file)

    # Compute PESQ
    pesq_score = compute_pesq(original_file, transferred_file, 16000)

    # STOI
    stoi_score = calculate_stoi(original_file, transferred_file)

    # MSE
    mse_score = calculate_mse(original_file, transferred_file)

    return {
        "MCD": mcd_value,
        "PESQ": pesq_score,
        "STOI": stoi_score,
        "MSE": mse_score
    }

def evaluate_files(original_file, transferred_files, row_names):
    results = []

    for i in range(len(transferred_files)):
        metrics = main(original_file, transferred_files[i])
        metrics['Speaker Encoder'] = row_names[i]  # Add the file name for reference
        results.append(metrics)

    # Create a DataFrame
    df = pd.DataFrame(results)
    df.set_index('Speaker Encoder', inplace=True)
    return df

In [23]:
file_location = 'wavs/p225_001/'
original_file = file_location+'p225_001.wav'
transferred_files = [file_location+'p225xp225_original.wav',
                     file_location+'p225xp225_facebook.wav',
                     file_location+'p225xp225_speechbrain.wav',
                     file_location+'convgru_p225xp225.wav']
row_names = ['Zero-Shot','Facebook','SpeechBrain','ConVGRU']

df = evaluate_files(original_file, transferred_files, row_names)
df

,MCD,PESQ,STOI,MSE
Speaker Encoder,,,,
Zero-Shot,190.872009,1.095009,0.160352,0.031352
Facebook,188.664474,1.057051,0.175409,0.024982
SpeechBrain,249.722000,1.046293,0.056638,0.018245
ConVGRU,169.612961,1.097576,0.576549,0.020387
